In [2]:
!pip install pandas

You should consider upgrading via the '/Users/tenzin/Desktop/mycodes/nba-draft-ranker/.venv/bin/python3 -m pip install --upgrade pip' command.


In [11]:
from pathlib import Path

# Get the notebook's directory
current_dir = Path.cwd()  # or Path(__file__).parent if running as .py script
draft_dir = current_dir.parent / "drafts"  # go one level up, then into drafts/

files = list(draft_dir.glob("draft_*.csv"))

df = pd.DataFrame()
for file in files:
    curr_df = pd.read_csv(file)
    df = pd.concat([df, curr_df])

# Sort dataframe by Year and then Overall Pick
df = df.sort_values(by=["SEASON", "OVERALL_PICK"])

df.to_csv(draft_dir / "drafts.csv", index=False)
